In [1]:
import numpy as np

## Creating base `Operation` class

In [2]:
class Operation():
    
    def __init__(self, input_nodes = []):
        
        self.input_nodes = input_nodes
        self.output_nodes = []
        
        for node in input_nodes:
            node.output_nodes.append(self)
            
        _default_graph.operations.append(self)
            
            
    def compute(self):
        pass

## Creating example operations

In [3]:
class add(Operation):
    
    def __init__(self, x, y):
        
        super().__init__([x, y])
        
        
    def compute(self, x, y):
        
        self.inputs = [x, y]
        
        return x + y

In [4]:
class multiply(Operation):
    
    def __init__(self, x, y):
        
        super().__init__([x, y])
        
        
    def compute(self, x, y):
        
        self.inputs = [x, y]
        
        return x * y

In [5]:
class matmul(Operation):
    
    def __init__(self, x, y):
        
        super().__init__([x, y])
        
        
    def compute(self, x, y):
        
        self.inputs = [x, y]
        
        return x.dot(y)

## Creating `Placeholder` class

In [6]:
class Placeholder():
    
    def __init__(self):
        
        self.output_nodes = []
        
        _default_graph.placeholders.append(self)

## Creating `Variable` class

In [7]:
class Variable():
    
    def __init__(self, initial_value = None):
        
        self.value = initial_value
        self.output_nodes = []
        
        _default_graph.variables.append(self)

## Creating `Graph` class

In [8]:
class Graph():
    
    def __init__(self):
        
        self.operations = []
        self.placeholders = []
        self.variables = []
        
        
    def set_as_default(self):
        
        global _default_graph
        _default_graph = self

## Creating `traverse_postorder` function

In [9]:
def traverse_postorder(operation):
    
    nodes_postorder = []
    
    def recurse(node):
        if isinstance(node, Operation):
            for input_node in node.input_nodes:
                recurse(input_node)
        nodes_postorder.append(node)
            
    recurse(operation)
    
    return nodes_postorder

## Creating `Session` class

In [10]:
class Session():
    
    def run(self, operation, feed_dict = {}):
        
        nodes_postorder = traverse_postorder(operation)
        
        for node in nodes_postorder:
            
            if type(node) == Placeholder:
                node.output = feed_dict[node]
            elif type(node) == Variable:
                node.output = node.value
            else:
                node.inputs = [input_node.output for input_node in node.input_nodes]
                node.output = node.compute(*node.inputs)
                
            if type(node.output) == list:
                node.output = np.array(node.output)
                
        return operation.output

## Example 

z = Ax + b

A = 10

b = 1

z = 10x + 1

In [11]:
# Single values

g = Graph()
g.set_as_default()

A = Variable(10)
b = Variable(1)

x = Placeholder()

y = multiply(A, x)
z = add(y, b)

sess = Session()
result = sess.run(operation = z, feed_dict = {x: 10})

print(result)

101


In [12]:
# Matrix values

g = Graph()
g.set_as_default()

A = Variable([[1, 2], [3, 4]])
b = Variable([1, 2])

x = Placeholder()

y = matmul(A, x)
z = add(y, b)

sess = Session()
result = sess.run(operation = z, feed_dict = {x: 10})

print(result)

[[11 22]
 [31 42]]
